# Sprint
## 深層学習スクラッチ リカレントニューラルネットワーク

## 1.このSprintについて

### Sprintの目的
- スクラッチを通してリカレントニューラルネットワークの基礎を理解する

### どのように学ぶか
スクラッチでリカレントニューラルネットワークの実装を行います。

## 2.リカレントニューラルネットワークスクラッチ

**リカレントニューラルネットワーク（RNN）**のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。


フォワードプロパゲーションの実装を必須課題とし、バックプロパゲーションの実装はアドバンス課題とします。


クラスの名前はScratchSimpleRNNClassifierとしてください。クラスの構造などは以前のSprintで作成したScratchDeepNeuralNetrowkClassifierを参考にしてください。

### 【問題1】
#### SimpleRNNのフォワードプロパゲーション実装
SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。


フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。


バッチサイズを`batch_size`、入力の特徴量数を`n_features`、RNNのノード数を`n_nodes`として表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。

$$
a_t = x_{t}\cdot W_{x} + h_{t-1}\cdot W_{h} + B\\
h_t = tanh(a_t)
$$

$a_t$ : 時刻tの活性化関数を通す前の状態 (batch_size, n_nodes)


$h_t$ : 時刻tの状態・出力 (batch_size, n_nodes)


$x_{t}$ : 時刻tの入力 (batch_size, n_features)


$W_{x}$ : 入力に対する重み (n_features, n_nodes)


$h_{t-1}$ : 時刻t-1の状態（前の時刻から伝わる順伝播） (batch_size, n_nodes)


$W_{h}$ : 状態に対する重み。 (n_nodes, n_nodes)


$B$ : バイアス項 (n_nodes,)


初期状態 $h_{0}$ は全て0とすることが多いですが、任意の値を与えることも可能です。


上記の処理を系列数n_sequences回繰り返すことになります。RNN全体への入力 $x$ は(batch_size, n_sequences, n_features)のような配列で渡されることになり、そこから各時刻の配列を取り出していきます。


分類問題であれば、それぞれの時刻のhに対して全結合層とソフトマックス関数（またはシグモイド関数）を使用します。タスクによっては最後の時刻のhだけを使用することもあります。



In [1]:
import numpy as np

In [12]:
class SimpleRNN:
    """
    RNN中間層
    Parameters
    ----------
    n_features : int
      RNNの入力の次元
    n_nodes : int
      RNNのノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス

    Attribute
    --------
    self.W_x : ndarray(n_features, n_nodes)
      入力に対応する重み
    self.W_h : ndarray(n_nodes, n_nodes)
      状態に対応する重み
    self.B : ndarray(n_node,)
      バイアス
    self.X_ : ndarray(batch_size, n_features)
      保存された入力
    """
    def __init__(self, n_features, n_nodes, initializer=None, lr=0.01):
        # 初期化
        # initializerのメソッドを使い、self.Wx, self.Wt, self.Bを初期化する
        if initializer is not None:
            self.W_x = initializer.W(n_features, n_nodes)
            self.W_h = initializer.W(n_nodes,    n_nodes)
            self.B   = initializer.B(n_nodes)
        self.lr = lr
        
    def forward(self, h_t, X):
        """
        フォワード
        Parameters
        ----------
        h_t : ndarray(batch_size, n_nodes)
            状態
        X : ndarray(batch_size, n_features)
            入力
        Returns
        ----------
        A : ndarray(batch_size, n_nodes)
            出力
        """
        self.X_ = X

        A = X@self.W_x + h_t@self.W_h + self.B

        return A

フォワードプロパゲーション部
```
    def forward(self, h_t, X):
        """
        フォワード
        Parameters
        ----------
        h_t : ndarray(batch_size, n_nodes)
            状態
        X : ndarray(batch_size, n_features)
            入力
        Returns
        ----------
        A : ndarray(batch_size, n_nodes)
            出力
        """
        self.X_ = X

        A = X@self.Wx + h_t@self.Xt + self.B

        return A
```

### 【問題2】
#### 小さな配列でのフォワードプロパゲーションの実験
小さな配列でフォワードプロパゲーションを考えてみます。


入力x、初期状態h、重みw_xとw_h、バイアスbを次のようにします。


ここで配列xの軸はバッチサイズ、系列数、特徴量数の順番です。

```python
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)
```

フォワードプロパゲーションの出力が次のようになることを作成したコードで確認してください。

```python
h = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]]) # (batch_size, n_nodes)
```

In [67]:
class Tanh:
    """
    tanh関数

    Attribute
    ----------
    self.A
        活性化関数の入力
    """
    def forward(self, A):
        self.A = A
        Z = (np.exp(A)-np.exp(-A)) / (np.exp(A)+np.exp(-A))
        
        return Z

In [59]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)

In [68]:
sampleRNN = []

# n_sequences方向にネットワーク作成
for i in range(n_sequences):
    sampleRNN.append([SimpleRNN(n_features=n_features, n_nodes=n_nodes), Tanh()])
    # 重みとバイアスの設定
    sampleRNN[i][0].W_x = w_x
    sampleRNN[i][0].W_h = w_h
    sampleRNN[i][0].B   = b

# 順伝搬
h_tmp = h
for j in range(n_sequences):
    # RNN層
    h_tmp = sampleRNN[j][0].forward(h_tmp, x[:, j, :])
    # 活性化関数
    h_tmp = sampleRNN[j][1].forward(h_tmp)
h_last = h_tmp

"""
最終的な出力の確認
output:
h = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]]) # (batch_size, n_nodes)
[[0.79494228, 0.81839002, 0.83939649, 0.85584174]] (1, 4)
"""
print(h_last, h_last.shape)

[[0.79494228 0.81839002 0.83939649 0.85584174]] (1, 4)


### 【問題3】
#### （アドバンス課題）バックプロパゲーションの実装
バックプロパゲーションを実装してください。


RNNの内部は全結合層を組み合わせた形になっているので、更新式は全結合層などと同様です。

$$
W_x^{\prime} = W_x - \alpha \frac{\partial L}{\partial W_x} \\
W_h^{\prime} = W_h - \alpha \frac{\partial L}{\partial W_h} \\
B^{\prime} = B - \alpha \frac{\partial L}{\partial B}
$$

$\alpha$ : 学習率


$\frac{\partial L}{\partial W_x}$ : $W_x$ に関する損失 $L$ の勾配


$\frac{\partial L}{\partial W_h}$ : $W_h$ に関する損失 $L$ の勾配


$\frac{\partial L}{\partial B}$ : $B$ に関する損失 $L$ の勾配


勾配を求めるためのバックプロパゲーションの数式が以下です。


$\frac{\partial h_t}{\partial a_t} = \frac{\partial L}{\partial h_t} ×(1-tanh^2(a_t))$


$\frac{\partial L}{\partial B} = \frac{\partial h_t}{\partial a_t}$


$\frac{\partial L}{\partial W_x} = x_{t}^{T}\cdot \frac{\partial h_t}{\partial a_t}$


$\frac{\partial L}{\partial W_h} = h_{t-1}^{T}\cdot \frac{\partial h_t}{\partial a_t}$


＊$\frac{\partial L}{\partial h_t}$ は前の時刻からの状態の誤差と出力の誤差の合計です。hは順伝播時に出力と次の層に伝わる状態双方に使われているからです。


前の時刻や層に流す誤差の数式は以下です。


$\frac{\partial L}{\partial h_{t-1}} = \frac{\partial h_t}{\partial a_t}\cdot W_{h}^{T}$


$\frac{\partial L}{\partial x_{t}} = \frac{\partial h_t}{\partial a_t}\cdot W_{x}^{T}$

In [ ]:
def backward(self, dLh, dLx):
    